#***Assignment 6: Taking the Pulse of the City: Who is impacted by air pollution in cities?***
This assignment is due on **2/15/2024**.
Change the name of your notebook to **tpp_assignment_6_sunetID.ipynb**
Share your completed notebook with the TAs akroo@stanford.edu & flora221@stanford.edu using the share banner at the top. If you are still having technical difficulties, email us before the deadline.



###**INTRODUCTION TO THE ASSIGNMENT**

This week’s assignment will get us thinking about environmental justice and aerosolized particulate matter pm2.5.


###**DATA SETS**
The dataset that we will be using today comes from the PurpleAir Sensor Network. These are low cost sensors, purchased primarily by individuals to monitor air quality for their personal decision making. Some communities and cities have also invested in these monitors for better understanding health risks and supporting regulatory decision making.

The dataset is over one year (2023), the sensors take measurements every few minutes, but the data has been down-sampled so that there are hourly data points that represent the average pm2.5 concentration measurement for that hour for that sensor.

These sensors have been deployed globally, however for the context of this assignment, we will be focusing on their data from across the bay in Oakland, CA. We have focused on Oakland for a few reasons. Oakland is one of the most diverse cities in the bay area in just about every equity metrics. Because of this, Oakland has a complex and deeply historical relationship with racial justice, being the one of the centers of civil rights movements from its founding, through the civil rights movement of the ‘60s to today. One of the implications of this is that the City of Oakland Department of Transportation has done a tremendous amount of work to make census data available to the general public and researchers looking to take a data driven approach to analyzing inequities and public policy effects through their Geographic Equity Tookbox (OGET).


####**Units:**
Micrograms per cubic meter of air



##**TOOLBOX**
All the Python functions and packages you will use in this assignment are in the toolbox for the course. We add new tools to the toolbox with each assignment as new ways of analyzing and visualizing data are introduced.

https://colab.research.google.com/drive/1gQxlpnogdJzykfNqjIGO4VLMnHHhrvcC?usp=drive_link




##**THE LEARNING GOALS FOR THE WEEK**
where the course learning goals are in plain text, and the focus this week is in italics.


- Become familiar with the wide range of sensors available to study various components of the Earth system. These include sensors on satellites, aircraft, ground-based platforms, and deployed above or beneath the surface on land or water.
-  Become familiar with the basic physical principles (resolution, sampling, processing workflows, etc.) common to all sensors.
- Work with various sources of data, learning how to access, analyze, synthesize, and describe the data to quantify trends; think critically and creatively about how to project these trends into the future.


#### 1) **Install and Import Packages**: numpy, pandas, matplotlib (See Toolbox)

In [ ]:
#Download important packages to runtime
!pip install geojson rioxarray pykrige &> /dev/null
!pip install cartopy &> /dev/null

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import geopandas as gpd
import geojson
import xarray as xr
import rioxarray
import cartopy.crs as ccrs
import cartopy.feature as cf
import shapely #useful in reshaping oget data
from shapely.geometry import mapping #useful
from pykrige.ok import OrdinaryKriging


#### 2) **Getting the Data**: Loading the datasets
Ensure that you have reviewed the Introduction section, especially the discussion of the dataset. Take 3 minutes to familiarize yourself with Oakland’s Geographic Equity Toolbox (OGET) available at oakgis.maps.arcgis.com. One of our datasets comes from this site and includes all of the data categories that their visualization displays.


In [ ]:
!git clone https://premonition.stanford.edu/taking-the-pulse-of-the-planet/OGET_data

Cloning into 'OGET_data'...
remote: Enumerating objects: 44, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 44 (delta 2), reused 0 (delta 0), pack-reused 36
Receiving objects: 100% (44/44), 62.90 MiB | 16.56 MiB/s, done.
Resolving deltas: 100% (15/15), done.
Updating files: 100% (12/12), done.


In [ ]:
oak = gpd.read_file('OGET_data/Priority_Neighborhoods/Priority_Neighborhoods.shx')
oak['coords'] = oak['geometry'].apply(lambda x: x.representative_point().coords[:])
oak['coords'] = [coords[0] for coords in oak['coords']]

df_oak = pd.read_csv('OGET_data/oak_2023.csv')
df_oak.rename(columns = {'datetime':'date'},inplace=True)
df_oak.drop(columns = 'Unnamed: 0',inplace=True)
df_oak = df_oak[df_oak.lon != df_oak.lon.max()]
df_oak = df_oak[df_oak.lon != df_oak.lon.max()]


#pull data
with open('OGET_data/GET_2021.geojson', 'r') as f:
  feature_collection = geojson.load(f)
#reformat data
all_features_data = pd.DataFrame()
for feature in feature_collection.features: # 117 features
  feature_geometry = feature.geometry # geojson geometry.
  feature_data = feature.properties # 92 columns!
  feature_id = feature_data['GEOID'] # they provide because it seems like a unique id
  feature_data_df = pd.DataFrame(feature_data, index=[feature_id])
  all_features_data = pd.concat([all_features_data, feature_data_df])


In [ ]:
import warnings
warnings.filterwarnings('ignore')
true_NW = (37.885367, -122.246913) #north west corner of neighborhood
true_SE = (37.849975, -122.214179) #south east corner of neighborhood
real_range = (true_NW[0]-true_SE[0], abs(true_NW[1]-true_SE[1])) #neighborhood range

known_obj = oak.loc[[94]] #selecting known neighborhood
minx_k, miny_k, maxx_k, maxy_k = known_obj.total_bounds #getting the bounds of the neighborhood object
fake_range = (maxx_k-minx_k, maxy_k-miny_k) #What the dataset thinks its x y ranges are
known_obj_center = 0.5*(maxx_k+minx_k), 0.5*(maxy_k+miny_k) #the true center of the object

scaling_factor = (real_range[0]/fake_range[0], real_range[1]/fake_range[1]) #how much do we need to zoom/unzoom
offset_factor = ((true_NW[1]+true_SE[1])/2)-known_obj_center[0]*scaling_factor[0],((true_NW[0]+true_SE[0])/2)-known_obj_center[1]*scaling_factor[1] #how much do we need to translate it

#transform each object saving the centroid, geometry, and geoid for each object
for i in range(len(oak)):
  test_obj = oak.loc[[i]]
  test_obj=test_obj.scale(scaling_factor[0],scaling_factor[1],origin=(0,0))
  test_obj=test_obj.translate(+offset_factor[0], +offset_factor[1])
  if i == 0:
    oak_scaled = gpd.GeoDataFrame()
    oak_scaled['geometry'] = test_obj
    oak_scaled['coords'] = test_obj.centroid
    oak_scaled['geoid'] = oak.loc[[i]].GEOID
  else:
    temp = gpd.GeoDataFrame()
    temp['geometry'] = test_obj
    temp['coords'] = test_obj.centroid
    temp['geoid'] = oak.loc[[i]].GEOID
    oak_scaled = pd.concat([oak_scaled,temp])
minx_tot, miny_tot, maxx_tot, maxy_tot = oak_scaled.total_bounds

#makes a geoseries containing the boarder of all of oakland without the neighborhood divisions, sometimes prettier
all_oak = gpd.geoseries.GeoSeries(shapely.ops.unary_union(oak_scaled.geometry.buffer(.0001)))


#### 3) **Guided Data Analysis**
Each sensor value represents the value of PM2.5 concentration(particulates under 2.5um) at a single point in space. As such, a natural way of displaying this data is with a scatter plot where every location of a sensor is a dot and the color of the dot corresponds to the quantity of PM2.5 in the air.

##### **a)** Using the toolbox, make a scatter plot map of the data at a single time including the outlines of the neighborhoods of Oakland.

Hint: break down this question into what you need to do to get the data at a single time and looking for a method to plot it in a scatter plot.


In [ ]:
# Your Scatter Plot Map Here

##### **b)** What do you notice about the distribution of sensors? Why do you think it is not uniform across the city? Use the Oakland Geographic Equity Toolbox’s map and look up information about Purple Air sensors and Purple Air’s business model to inform your answer.

https://www.oaklandca.gov/resources/oakdot-geographic-equity-toolbox

**Answer:**

##### **c)** Make a scatter plot map of the average/mean pm2.5 concentration across the city

We have included the code to take the average with respect to time of PM2.5 concentration. This means we are left with a single scatter plot map-like data frame in df_oak_avg


In [ ]:
df_oak_avg = df_oak.groupby(['lat','lon'], as_index=False).mean() #group data to have lat/lon/values divided up by date

In [ ]:
# Your Scatter Plot Map Here

In your plot from Part C, you may notice a lot of close together data points that give drastically different values. Because we are dealing with a dataset where the sensors are not actively managed by anyone, there can be a lot of variability in how they are set up and used. For example, all of the sensors included claim that they are outside, but there is no verification that the characteristics that the user has input are correct.


##### **d)** Using the toolbox filter out the average data in the extremes (below 25% and above 75%) and visualize it in a scatter plot map.

In [ ]:
# Your Data Filtering and Scatter Plot Map Here

In the last part we have reduced the impact of wonky sensors by eliminating spatial datapoints that are outliers, however outliers in these sensors exist between sensors (like in part d) as well as over time. On approach to mitigating the impact of these temporal outliers is to take the median instead of the mean of our data


##### **e)** Find the median pm2.5 concentrations at each sensor, filter the data to exclude wonky sensors (exclude extremes below 17% and above 95%). Plot it as a scatter plot map.

In [ ]:
# Your Median, Filtered Data, Scatter Plot Here

##### **f)** Why might a mean or median be an appropriate measure for excluding outliers in this case? (2-5 sentances)


**Answer:**

------

While these scatter plots help us describe the spatial distribution of pm2.5, there is still a substantial amount of variation in values between sensors that makes it hard to pick out and quantify spatial patterns. To address this, we will interpolate between the data points using a method used by air quality community called Kriging. Below is the function that we will use to do the interpolation, run the cell to create the function.


In [ ]:
# Defines function that we will use to interpolate our air quality in 2D
def kriging(df,minx=-122.3506,maxx=-122.1126,miny=37.7086,maxy=37.8840,vari_mod = "hole-effect",cells=100):
  '''
  df: Data frame input should have four columns "date",	"value",	"lat",	"lon"
  minx,...maxy: min and max values for final interpolated map
  Optional input: variogram model type. Options are linear, power, gaussian, spherical, exponential and hole-effect
  '''
  # Dropping all nan values for nan handling
  values = df['value']
  inds_active = ~np.isnan(values)
  xold = df['lon'][inds_active]
  yold = df['lat'][inds_active]
  zold = df['value'][inds_active]

  # Create a 100 by 100 grid
  # Horizontal and vertical cell counts should be the same
  XX_pk_krig = np.linspace(minx, maxx, cells)
  YY_pk_krig = np.linspace(miny, maxy, cells)

  # Generate ordinary kriging object
  OK = OrdinaryKriging(xold,yold,zold,variogram_model = vari_mod,verbose = False,enable_plotting = False,coordinates_type = "euclidean",exact_values=False)

  # Evaluate the method on grid
  Z_pk_krig, sigma_squared_p_krig = OK.execute("grid", XX_pk_krig, YY_pk_krig)
  correct_dims = ['lat', 'lon']
  correct_coords = {'lat': YY_pk_krig, 'lon': XX_pk_krig}
  correct_ds = xr.Dataset({'pm25': (correct_dims, Z_pk_krig)}, coords=correct_coords)
  return correct_ds

##### **g)** Using the toolbox, interpolate the median scatter plot map data from part E.


In [ ]:
# Your Interpoplated Median Plot Here

##### **h)** Does this interpolated plot match what you expected? Are there areas of your interpolation that are more or less reliable? Why?

**Answer:**

------
Now that we have a gridded xarray dataset, we are finally in a position to start to answer our guiding question -- **Who is impacted by air pollution in cities?**

By modifying the country_clipping function that went through and clipped all of the CO2 data to each country in Assignment 3, we can do the same thing for neighborhoods’ PM2.5 concentration. We get out a list containing the average pm2.5 concentration from our interpolated data for each neighborhood.

The code cell below lists all of the data categories in the Oakland Geographic Equity Toolbox, use their website to understand what each category means: https://www.oaklandca.gov/resources/oakdot-geographic-equity-toolbox.


In [ ]:
# Function for determining average pm25 over each neighborhood
# You do not need to understand this now, we will revisit it in the next assignment
# Should be familiar...
def neighbor_clipped_averages(dataset,bounds):
    '''Function that takes in dataset and gdf_boundaries and returns dataframe
    of average value of pm25 over that neighbor.
    Example of use: neighbor_vals = neighbor_clipped_averages(pm25_dataset, gdf_boundaries)'''
    neighbor_vals = pd.DataFrame(index=bounds.index)
    neighbor_vals.pm25 = 0
    neighbor_names = bounds.index
    for neighbor in neighbor_names:
      data_copy = dataset
      neighbor_boundary = bounds.loc[[neighbor]]
      data_copy.rio.set_spatial_dims(x_dim="lon", y_dim="lat", inplace=True)
      data_copy.rio.write_crs(3857, inplace=True)
      data_clipped = data_copy.rio.clip(
          neighbor_boundary.geometry.apply(mapping),
          neighbor_boundary.crs,
          drop=True
      )

      neighbor_vals.at[neighbor, 'pm25'] = float(data_clipped.pm25.mean(dim=['lon','lat']))
    return neighbor_vals

##### **i)** Plot neighborhood air quality as a function of a metric of your choice from the Oakland Geographic Equity categories.

In [ ]:
# this will return all the columns in our all features data
# Use this list to find a neighborhood metric you want to explore
# Ignore the first 9 options!
all_features_data.columns

Index(['OBJECTID', 'NAME', 'FID_TL_201', 'TRACTCE', 'GEOID', 'NAME_1',
       'NAMELSAD', 'GEOID2_1', 'GEOGRAPHY', 'TOTAL_POP', 'TOTAL_POP_',
       'TOTAL_POP1', 'RACE_TOTAL', 'TOTAL_PO_1', 'TOTAL_PO_2', 'ONE_RACE',
       'ONE_RACE_W', 'BLACK_OR_A', 'AMINDIANAN', 'AMINDIAN_1', 'AMINDIAN_2',
       'AMINDIAN_3', 'AMINDIAN_4', 'ASIAN', 'ASIAN_INDI', 'ASIAN_CHIN',
       'ASIAN_FILI', 'ASIAN_JAPA', 'ASIAN_KORE', 'ASIAN_VIET', 'ASIAN_OTHE',
       'NATIVE_HI_', 'NATIVE_HI1', 'NATIVE_H_1', 'NATIVE_H_2', 'NATIVE_H_3',
       'OTHER', 'TWO_OR_MOR', 'WHITE_AND_', 'WHITE_AND1', 'WHITE_AN_1',
       'BLACK_AND_', 'HISPLAT_TO', 'HISPLAT__1', 'HISPLAT__2', 'HISPLAT__3',
       'HISPLAT__4', 'HISPLAT__5', 'HISPLAT_NO', 'HISPLAT_WH', 'HISPLAT_BL',
       'HISPLAT_AM', 'HISPLAT_AS', 'HISPLAT_NA', 'HISPLAT_OT', 'HISPLAT_2O',
       'HISPLAT__6', 'HISPLAT_2_', 'HISPLAT_HO', 'VOTERS_OVE', 'VOTERS_O_1',
       'VOTERS_O_2', 'SHAPE_LENG', 'SHAPE_AREA', 'SHAPE_AR_1', 'SHAPE_LEN',
       'TractNumb', 'Tra

In [ ]:
# TO USE: PASTE COLUMN NAME FROM ABOVE INTO YOUR_METRIC_HERE
metric = all_features_data.YOUR_METRIC_HERE # pull neighborhood metric for demographics

#reformat some things. Don't worry about this
metric_dict = {all_features_data.GEOID[i]:metric[i] for i in range(len(all_features_data.GEOID))}
metric_list = []
geoids = oak_scaled.geoid
for x in geoids:
  metric_list.append(metric_dict[x])

gdf = gpd.GeoDataFrame({'geometry': oak_scaled.geometry, 'values': metric_list},
    crs="EPSG:3857")
gdf.set_geometry('geometry',inplace=True)

In [ ]:
# YOUR Plot of Neighborhood Metric vs PM2.5 Here

On this graph, we are interested in how closely the data fits a trend line, another way of saying this is *how correlated are these metrics*. We can calculate this numerically with the code below. A correlation coefficient is a number between -1 and 1 that represents how well a linear fit represents your data. A correlation of 1 would occur when your data falls perfectly along the line, -1 would mean it was perfectly on a line with the opposite slope to the trend line in comparison. With fewer data points, it's easier to make a line of best fit that closely approximates your data. This means we have to check to see if our correlation is statistically significant given the number of data points that we have as is shown in the second code block below.

##### **j)** Test if this correlation between our independent and dependent variables is statistically significant by running the following code. Then discuss the implications and socio-political-economic reasons for your findings

In [ ]:
pm25_values = YOUR_neighbor_vals_med.to_numpy().flatten() # THIS SHOULD BE THE OUTPUT OF THE neighbor_clipped_averages FUNCTION
metric_list = metric_list # this will correspond to the census metric defined earlier
r = np.corrcoef(pm25_values,y=metric_list)[0][1] # this gives us a numerical correlation betewen your metric and the air quality for all of the neighborhoods
r # Correlation coefficient

0.469173092207749

In [ ]:
# TEST OF SIGNIFICANCE:
r >= 2/np.sqrt(len(pm25_values))

True

**Answer:**

##**After Class**


#### **a)** Discuss red-lining and the continued impact it has had on the population of Oakland from a health, economic, and social lens. Feel free to expand beyond this limited framing.  (~10 sentences)


**Answer:**

#### **b)** What can be done and what is being done from a regulatory perspective to reduce the inequities that we have explored in this assignment?

**Answer:**

#### **c)** Given your exploration of the data, pose a question that you think the data can answer.

**Answer:**

#### **d)** Design a workflow (bullet points of steps for how you will answer the question)

**Answer:**
- List

#### **e)** Answer your question with code

In [ ]:
# Your Code Here

#### **f)** Discuss the implications of your question.

**Answer:**